<a href="https://colab.research.google.com/github/avdlaan/dissertation/blob/main/Read_Data_functions_from_Data_set_with_Ktrain_Question_and_Answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
%%capture
!pip3 install ktrain==0.28.2
!pip install tensorflow==2.4.1

In [11]:
import numpy as np
import pandas as pd 
import glob
import json
import math

Set the initial imports- for the next step let us iterate the directory.

In [12]:
all_json = glob.glob('/content/drive/MyDrive/train/document_parses/json_test/*.json', recursive=True)

# let us get the list of articles
len(all_json)
print(len(all_json))

39


In [13]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:400]}... {self.body_text[:500]}...'

In [14]:
first_row = FileReader(all_json[0])
print(first_row)

000a0fc8bbef80410199e690191dc3076a290117: Background: Various transcription factors are involved in the process of mutually exclusive expression and clonal variation of the Plasmodium multigene (var) family. Recent studies revealed that a P. falciparum SWI/SNF-related matrix-associated actin-dependent regulator of chromatin (PfSWIB) might trigger stage-specific programmed cell death (PCD), and was not only crucial for the survival and dev... Plasmodium falciparum, which causes malignant malaria such as cerebral malaria (CM) or pregnancy-associated malaria (PAM) [1, 2] , is the cause of death to 435,000 individuals annually [3] . Plasmodium falciparum erythrocyte membrane protein 1 (PfEMP1), which is encoded by the var gene family comprising approximately 60 members, is the major virulence factor involved in the antigenic variation and clinical pathogenicity of falciparum malaria [4, 5] . It is remarkable that the expression of var f...


In [15]:
dict_ = {'paper_id': [], 'abstract': [], 'body_text': []}
for idx, entry in enumerate(all_json):
    try:
        content = FileReader(entry)
        if (idx % 1000 == 0):
            print(f"processing {idx} of {len(all_json)} ")
        dict_['paper_id'].append(content.paper_id)
        dict_['abstract'].append(content.abstract)
        dict_['body_text'].append(content.body_text)
    except:
        pass
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text'])
#df_covid.head(39)

processing 0 of 39 


In [16]:
#df_covid = ae.extract(df_covid.body_text.values, df_covid, [('Covid Recovery?', 'Recovery'),
#                                     ('Long Covid signs?','Signs')])
#df_covid.head(12)

In [18]:
import ktrain
from ktrain.text.qa import AnswerExtractor

ae = AnswerExtractor()

df_covid_output = ae.extract(df_covid.body_text.values, df_covid, [('What proteins make up the Covid Virus?', 'Proteins in Covid')],return_conf=True)

df_covid_output.head(39)


df_covid_output.to_excel('/content/drive/MyDrive/train/qa_file2.xlsx',sheet_name="Sheet1",startrow=0,startcol=0,merge_cells=True)
